## Importation des bliblothèques

In [35]:
import sqlite3
import pandas as pd

## Connection à la base de données

In [36]:
connect = sqlite3.connect("../../Databases/raw-database.db")
cursor = connect.cursor()

## Importation des données freeze frames

In [37]:
req = cursor.execute(f"SELECT * FROM freeze_frames")
res = req.fetchall()
desc = req.description
freeze_frames = pd.DataFrame(res)
freeze_frames.columns = [i[0] for i in desc]

In [38]:
# Aperçu des données
freeze_frames.head(3)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object
0,4820,2024-12-06 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN
1,4834,2024-12-06 00:04:30.400000,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,1,1020089,away team,159756.0
2,4854,2024-12-06 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN


In [39]:
# On récupère les event_id SB pour pouvoir importer uniquement ceux qui nous intéressent depuis la table events
event_id_freeze_frames = freeze_frames.event_id.unique().tolist()

## Importation des données events

In [40]:
req = cursor.execute(f"SELECT event_id, pass_cross, pass_type FROM events WHERE event_id IN ({', '.join('?' * len(event_id_freeze_frames))})",
                     event_id_freeze_frames)
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [41]:
# Aperçu des données
events.head(3)

,event_id,pass_cross,pass_type
0,872df8f3-a750-4b61-8890-61a35e96b4a4,1.0,None
1,102a8d71-103b-4a16-a16a-09315649df86,1.0,None
2,6bf75176-f3ab-4e0a-8110-a5790be58b16,1.0,None


## Jointure des events avec les freeze frames

In [42]:
df = pd.merge(freeze_frames, events, on = "event_id")
df.sort_values(by = ["match_id_SKC", "timestamp"], inplace = True)

In [43]:
# Aperçu des données
df.head(3)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,pass_cross,pass_type
0,4820,2024-12-06 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,NaN,None
1,4834,2024-12-06 00:04:30.400000,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,1,1020089,away team,159756.0,1.0,None
2,4854,2024-12-06 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN,NaN,None


## Analyse des centres

In [50]:
# Création d'un dataframe ne comprenant que les centres du dataframe initial
centres = df[df.pass_cross == 1]

In [51]:
# On regarde si il y'a des valeurs centres qui sont présents plusieurs fois
# On utilise l'identifiant ["frame", "match_id_SKC"] qui est unique à chaque freeze_frame
centres[["frame", "match_id_SKC"]].duplicated().any()

np.False_

On souhaite maintenant identifier chaque centre et chaque évènement précédent et suivant un centre par un centre_id.  
Chaque couple (event précédent le centre, centre, event suivant le centre) sera alors identifié.  
De plus, on part du principe qu'on dispose bien de chaque évènements précèdents et suivant les centres.  
Il est donc notamment nécessaire des supprimes les events correspondants à des coups de pied arretés.

In [48]:
# Liste des CPA définis par Stats Bomb
CPA = ["Corner", "Free Kick", "Goal Kick", "Kick Off"]

# Supression des CPA des centres
centres = centres[~centres.pass_type.isin(CPA)]

In [53]:
# identification des évènements précédents et suivants les centres
index_events_avant_centre = centres.index - 1
index_events_apres_centre = centres.index + 1

In [54]:
# Intersection entre les index des évènements précédents et suivants les centres
index_events_avant_centre.intersection(index_events_apres_centre)

Index([  875,   901,  2124,  3212,  3712,  5559,  6161,  6559,  7902,  9338,
        9550,  9852, 10046, 10540, 10545, 10586, 11416, 13395, 13484, 13843,
       13944, 15251, 15835, 16779, 17114, 17776, 17937, 19640],
      dtype='int64')

On remaque qu'il y a des events qui à la fois précèdent et suivent des centres

In [14]:
# Création d'un dataframe ne comprenant que les actions précédents les centres
events_avant_centre = df.loc[centres.index - 1]

Pas de message d'erreur (index out of range), il y'a donc bien un event précédent chaque centre

In [15]:
# On regarde si il y'a des évènements précédents des centres qui correspondent à des centres
events_avant_centre[events_avant_centre.pass_cross == 1]

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,pass_cross,type


Il n'y en a pas

In [13]:
req = cursor.execute(f"SELECT * FROM freeze_frames_data")
res = req.fetchall()
desc = req.description
freeze_frames_data = pd.DataFrame(res)
freeze_frames_data.columns = [i[0] for i in desc]

In [14]:
freeze_frames_data.head(5)

,track_id,trackable_object,is_visible,x,y,vx,vy,speed_norm,accel_norm,z,frame,match_id_SKC
0,12235.0,12235.0,1,45.43,-0.97,2.594984,0.087762,2.596468,-0.145596,NaN,4820,1020089
1,12327.0,12327.0,1,32.26,-2.78,5.386337,-0.666400,5.427404,0.198751,NaN,4820,1020089
2,2340.0,2340.0,1,33.55,-16.15,5.579671,-0.015822,5.579693,0.400358,NaN,4820,1020089
3,12788.0,12788.0,1,30.18,5.24,6.059448,-1.084029,6.155650,0.328704,NaN,4820,1020089
4,25539.0,25539.0,1,30.43,-24.45,5.782475,-0.472854,5.801777,2.292212,NaN,4820,1020089


In [15]:
len(events_avant_centre)

6720

In [16]:
df2 = pd.merge(events_avant_centre, freeze_frames_data, on = ["match_id_SKC", "frame"])

In [17]:
df2.head(5)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,...,track_id,trackable_object,is_visible,x,y,vx,vy,speed_norm,accel_norm,z
0,4820,2024-12-06 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,...,12235.0,12235.0,1,45.43,-0.97,2.594984,0.087762,2.596468,-0.145596,NaN
1,4820,2024-12-06 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,...,12327.0,12327.0,1,32.26,-2.78,5.386337,-0.666400,5.427404,0.198751,NaN
2,4820,2024-12-06 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,...,2340.0,2340.0,1,33.55,-16.15,5.579671,-0.015822,5.579693,0.400358,NaN
3,4820,2024-12-06 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,...,12788.0,12788.0,1,30.18,5.24,6.059448,-1.084029,6.155650,0.328704,NaN
4,4820,2024-12-06 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,...,25539.0,25539.0,1,30.43,-24.45,5.782475,-0.472854,5.801777,2.292212,NaN


In [18]:
len(df2)

171459